#### ASTR 3890 - Selected Topics: Data Science for Large Astronomical Surveys (Spring 2022)
***N. Hernitschek***
___

# Homework 5
### Due: Monday, Feb 28th at 11.00am CST

The completed lecture notebook must also be submitted by 11:00am Central Time on Feb 28th.

---

## Problem 1

We have seen that for some distributions, we can solve the for the maximum likelihood analytically.

a) Solve analytically for the maximum likelihood of the Poisson distribution.

b) Solve analytically for the maximum likelihood for a Gaussian where the uncertainties are *hetero*scedastic.

Either write your solution as Markdown/Latex below, or paste in a scanned image of your handwritten solution.

### Solution

## Problem 2

In the lecture we have seen how data sets influenced by outliers can be fitted better when using the Huber loss function.
Another way to deal with outliers is called Winsorizing or winsorization and implemented in scipy (from scipy.stats.mstats import winsorize).
a) using the scipy online documentation, look up what this method means and write a small Python example.
b) try to fit the data from our lecture, section "Fitting A Straight Line To Data", after applying Winsorizing, and describe 



### Solution



In [30]:
#Python example of winsorizing
#From https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mstats.winsorize.html

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize

a = np.array([10, 4, 9, 8, 5, 3, 7, 2, 1, 6])
print(a)

winsorize(a, limits=[0.1, 0.2])


[10  4  9  8  5  3  7  2  1  6]


masked_array(data=[8, 4, 8, 8, 5, 3, 7, 2, 2, 6],
             mask=False,
       fill_value=999999)

In [31]:
#Fitting a Straight Line to Data with Winsorizing
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

import scipy.stats
from scipy.stats import norm
from scipy.stats import uniform
from scipy import optimize
from astroML import stats as astroMLstats
from astroML.datasets import fetch_hogg2010test

In [32]:
#------------------------------------------------------------
# Get data from AstroML: this includes outliers
data = fetch_hogg2010test()
x = data['x'] # x data
y = data['y'] # y data
x_win = winsorize(x, limits=[0.1, 0.2])
y_win = winsorize(y, limits=[0.1, 0.2])

dy = data['sigma_y'] # uncertainties on y data

# Define the standard squared-loss function.
# This is just another name for chi^2
def squared_loss(m, b, x_win, y_win, dy):
    y_fit = m * x_win + b
    return np.sum(((y_win - y_fit) / dy) ** 2, -1)

# define a lambda function that defines the sum of squared errors.

# let's first exclude the outliers by chopping off the first 4 points.
f_squared = lambda beta: squared_loss(beta[0], beta[1], 
                                      x_win=x_win[4:], y_win=y_win[4:], 
                                      dy=dy[4:])

#------------------------------------------------------------
# compute the maximum likelihood 
beta0 = (1, 30) # initial guess for a and b
beta_squared = optimize.fmin(f_squared, beta0)

Optimization terminated successfully.
         Current function value: 16.623053
         Iterations: 70
         Function evaluations: 133


In [33]:
#------------------------------------------------------------
# Plot the results
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)

# plot the data
ax.errorbar(x[4:], y[4:], dy[4:], 
            fmt='.k', lw=1, ecolor='gray')

# plot the best fit linear relationship
x_fit = np.linspace(0, 350, 10)
ax.plot(x_fit, beta_squared[0] * x_fit + beta_squared[1], 
        ls='--', color='k',
        label="squared loss:\n $y=%.2fx + %.1f$" % tuple(beta_squared))

# plot the data (winsorize)
ax.errorbar(x_win[4:], y_win[4:], dy[4:], 
            fmt='.k', lw=1, ecolor='gray')

# plot the best fit linear relationship (winsorize)
x_fit_win = np.linspace(0, 350, 10)
ax.plot(x_fit_win, beta_squared[0] * x_fit_win + beta_squared[1], 
        ls='--', color='k',
        label="winsorize" % tuple(beta_squared))

ax.set_xlim(0, 350)
ax.set_ylim(100, 700)
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.legend(loc=4, prop=dict(size=14))

plt.show()

SyntaxError: invalid syntax (Temp/ipykernel_15372/3289834668.py, line 27)

I am not sure why the graph is not plotting but winsorizing the fetched x and y data will cause the data to have the 10% of the lowest value and the 20% of the highest values replaced. In doing so, outliers on the ends of the plotted line would be removed, allowing for a better fitted straight line.